In [216]:
import sys
import io
import os.path
import re
import tarfile
from datetime import datetime
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath

from pprint import pprint

import smart_open


documents_path = ".\\psla_data\\PSLAreduced.csv"
vocab_path = ".\\psla_data\\PSLAwordseries.csv"

# model parameters
num_topics = 30
num_docs = 0
num_words = 0
chunksize = 2000
passes = 100
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

#docs = []
#bow = []     # arrray of bow for doc, used to get probability

#docs_per_timeslice = []

#tokentoword = {}  # used to visualize the results of the model

# as the model's vocab list is not in the same order as our predefined counts we will create these look up tables
# how to use:
#    use the timeslicevocabcounts list to run the ITMTF iteration  
#      (index on the interation we are running)
#      (the ITMFT algorithm will use the index of the count to represent each word)
#    the ITMTF iteration will return new topic(s) using the index of the count array as a representative for the word 
#    use the index to find the vocabulary word
#    use the index to find the model token with wordindextotoken
#vocabtowordindex = {}     # dict of the preped vocabulary words to INDEX used to create wordindextotoken
#wordindextotoken = {}     # will be used to get the model's token after each ITMFT iteration
#timeslicevocabcounts = [] # an array of timeslices, each element contains an array of word counts for that timeslice


In [217]:

# load the cleansed data into an array of docs

docs = []
with open(documents_path) as swf:
    docs_per_timeslice = []
    count = 0
    curtimeslice = "2000.7.1"
    tempslice.append(curtimeslice)
    curdocs = []
    firstime = 0
    for line in swf:
        cells = line.split(',')
        docslice = cells[0] + "." + cells[1] + "." + cells[2]
        if firstime == 0 :
            firstime = 1
            curtimeslice = docslice
        if docslice != curtimeslice :
            curtimeslice = docslice
            docs_per_timeslice.append(curdocs)
            curdocs = []
        curdocs.append(count) 
        count += 1
        
        docs.append(cells[3])
    print(curdocs)
    docs_per_timeslice.append(curdocs)
swf.close
print('Number of time slices with docs: %d' % len(docs_per_timeslice))

# load the cleansed data into an array of docs
header = 0
timeslicevocabcounts = []
vocabtowordindex = {}
with open(vocab_path) as vwf:
    for line in vwf:
        linenumber = 1 # skip the header row
        cells = line.split(',')
        if header == 0 :
            header = 1
            i = 1 # skip header column
            while i < len(cells) - 1:  # the cleansing process adds a black cell at the end
                vocabtowordindex[cells[i]] = i-1  
                #print(cells[i])
                i += 1  
        else :
            wordcount = []
            i = 1 # skip header column
            while i < len(cells)-1:  # the cleansing process adds a black cell at the end
                wordcount.append(cells[i])  # create an array of vocab counts at this timeslice
                i += 1 
            timeslicevocabcounts.append(wordcount)
            
vwf.close
print('Number of time slices: %d' % len(timeslicevocabcounts))
print('Number of time vocab: %d' % len(vocabtowordindex))

"""

"""
doctokens = [doc.split() for doc in docs]
dictionary = Dictionary(doctokens)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in doctokens]
#print (corpus)

print('Number of documents: %d' % len(corpus))

# lets make dict look up tables so we can take the word back from the iteration and find the dict index
i = 0
wordindextotoken = {}
while i < len(dictionary):  
    wordindextotoken[dictionary[i]] = i
    tokentoword[i] = dictionary[i]
    i += 1
print('Number of unique tokens: %d' % len(wordtotoken))

print(wordindextotoken[dictionary[0]])
print(tokentoword[0])


[2644, 2645, 2646, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672]
Number of time slices with docs: 123
Number of time slices: 123
Number of time vocab: 12517
Number of documents: 2673
Number of unique tokens: 0
0
acted


In [218]:
"""

"""
doctokens = [doc.split() for doc in docs]
dictionary = Dictionary(doctokens)
bow = []
# Bag-of-words representation of the documents.
for doc in doctokens :
    bow.append(dictionary.doc2bow(doc))
    
    
corpus = [dictionary.doc2bow(doc) for doc in doctokens]
#print (corpus)

num_docs = len(corpus)
print('Number of documents: %d' % len(corpus))

# lets make dict look up tables so we can take the word back from the iteration and find the dict index
i = 0
while i < len(dictionary):  
    wordindextotoken[vocabtowordindex[dictionary[i]]] = i
    i += 1
tokentoword = dictionary.id2token
num_words = len(tokentoword)
print('Number of unique tokens: %d' % len(tokentoword))


Number of documents: 2673
Number of unique tokens: 12517


In [219]:
# Make a index to word dictionary.
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

model = LdaModel(
    corpus=corpus,
    id2word=tokentoword,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

print('Model Finished')
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 16:59:44
Model Finished
Current Time = 17:08:30


In [175]:
# Save model to disk.
temp_file = datapath("baseline_model")
model.save(temp_file)

In [220]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

pprint(top_topics)

Average topic coherence: -3.9388.
[([(0.048668746, 'bush'),
   (0.044677403, 'gore'),
   (0.017947802, 'voters'),
   (0.0140112955, 'percent'),
   (0.013177402, 'nader'),
   (0.011228038, 'vote'),
   (0.010083657, 'george'),
   (0.009815974, 'president'),
   (0.009045774, 'al'),
   (0.00838208, 'polls'),
   (0.007822583, 'campaign'),
   (0.007794866, 'poll'),
   (0.0077638966, 'vice'),
   (0.0075520854, 'gov'),
   (0.0073198033, 'democrats'),
   (0.007266987, 'democratic'),
   (0.0066389567, 'race'),
   (0.006243486, 'election'),
   (0.00606016, 'party'),
   (0.0058441753, 'republican')],
  -0.9102080203600448),
 ([(0.02676427, 'gore'),
   (0.020196639, 'president'),
   (0.020048749, 'clinton'),
   (0.014352959, 'bush'),
   (0.013953719, 'campaign'),
   (0.009459463, 'convention'),
   (0.008998704, 'vice'),
   (0.007820231, 'gores'),
   (0.006371964, 'speech'),
   (0.0062660277, 'time'),
   (0.005532202, 'al'),
   (0.0055044973, 'republican'),
   (0.0054681255, 'political'),
   (0.0049

In [177]:
newmodel = LdaModel.load(temp_file)

In [221]:
topics = model.get_topics()
#print(topics[0])


count = 0
newtopics = []
for topic in topics :
    newtopics.append(topic)
    count += 1
#print(count)

#lets add a new topic
newprob = []
i = 0
while i < 12517 :
    newprob.append(0)
    i += 1
newprob[0] = .5
newprob[1] = .5
print(tokentoword[0])
print(tokentoword[1])
newtopics.append(newprob)

num_topics += 1
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
model = LdaModel(
    corpus=corpus,
    id2word=tokentoword,
    chunksize=chunksize,
    alpha='auto',               
    eta=newtopics,                 # preset topic/word
    iterations=iterations,
    num_topics=num_topics,       # added a topic
    passes=passes,
    eval_every=eval_every
)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

acted
addresses
Current Time = 17:14:46
Current Time = 17:37:13


In [222]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

pprint(top_topics)

Average topic coherence: -2.3985.
[([(0.047337126, 'gore'),
   (0.027736008, 'president'),
   (0.022932861, 'campaign'),
   (0.016757553, 'clinton'),
   (0.013722479, 'vice'),
   (0.010718986, 'al'),
   (0.010018481, 'house'),
   (0.009100582, 'democratic'),
   (0.009063316, 'democrats'),
   (0.007937636, 'gores'),
   (0.00792463, 'republican'),
   (0.0070301844, 'senator'),
   (0.006753081, 'republicans'),
   (0.0065148915, 'white'),
   (0.006375032, 'presidential'),
   (0.0060948683, 'bush'),
   (0.0056694243, 'gov'),
   (0.0056345826, 'political'),
   (0.0055851615, 'former'),
   (0.0054341597, 'election')],
  -0.9164662612696478),
 ([(0.03742052, 'gore'),
   (0.02182368, 'bush'),
   (0.01854268, 'voters'),
   (0.016292835, 'convention'),
   (0.012796229, 'clinton'),
   (0.012072736, 'issues'),
   (0.010972528, 'president'),
   (0.009475273, 'democrats'),
   (0.0076529593, 'republican'),
   (0.007473797, 'gores'),
   (0.0071788197, 'speech'),
   (0.006769085, 'republicans'),
   (0.0

   (0.01299332, 'support'),
   (0.010938968, 'critics'),
   (0.010419484, 'refuge'),
   (0.010374903, 'national'),
   (0.009688791, 'drilling'),
   (0.009237389, 'office'),
   (0.008958852, 'calls'),
   (0.008941409, 'arctic'),
   (0.008789877, 'policy')],
  -3.1778712396254334),
 ([(0.04305491, 'page'),
   (0.03476092, 'front'),
   (0.024577403, 'faith'),
   (0.023318393, 'al'),
   (0.022560298, 're'),
   (0.020970825, 'gorey'),
   (0.020796452, 'aug'),
   (0.020007072, 'god'),
   (0.017221097, 'religious'),
   (0.0134313265, 'details'),
   (0.011507659, 'george'),
   (0.011482901, 'action'),
   (0.010081001, 'christian'),
   (0.009648785, 'course'),
   (0.009219245, 'article'),
   (0.008964388, 'gore'),
   (0.008395675, 'dont'),
   (0.008391273, 'jew'),
   (0.007852859, 'misstated'),
   (0.007816128, 'column')],
  -5.052980485265384),
 ([(0.041861203, 'black'),
   (0.033651117, 'bush'),
   (0.017810265, 'george'),
   (0.014049343, 'won'),
   (0.01382356, 'capital'),
   (0.011996108, 

In [223]:
# Save model to disk.
temp_file = datapath("it1_model")
model.save(temp_file)
model.show_topic(30, 50)



[('page', 0.043054912),
 ('front', 0.034760922),
 ('faith', 0.024577405),
 ('al', 0.023318395),
 ('re', 0.0225603),
 ('gorey', 0.020970827),
 ('aug', 0.020796454),
 ('god', 0.020007074),
 ('religious', 0.017221099),
 ('details', 0.013431327),
 ('george', 0.01150766),
 ('action', 0.011482902),
 ('christian', 0.010081002),
 ('course', 0.009648786),
 ('article', 0.009219246),
 ('gore', 0.008964389),
 ('dont', 0.008395676),
 ('jew', 0.008391274),
 ('misstated', 0.00785286),
 ('column', 0.007816129),
 ('sept', 0.007413629),
 ('oped', 0.0071716174),
 ('values', 0.00710866),
 ('affirmative', 0.007028318),
 ('political', 0.00672167),
 ('brown', 0.006506817),
 ('read', 0.0064676045),
 ('own', 0.0063592787),
 ('people', 0.0062711975),
 ('goremccain', 0.006228215),
 ('driving', 0.0062280907),
 ('moral', 0.006222769),
 ('creation', 0.005648522),
 ('previous', 0.0055184653),
 ('character', 0.0052555003),
 ('car', 0.005189169),
 ('nature', 0.005154712),
 ('master', 0.005021244),
 ('book', 0.00489732

In [224]:
#temp_file = datapath("baseline_model")
#newmodel = LdaModel.load(temp_file)
#newmodel.show_topic(29, 50) # will blow up if 30

In [229]:
# create a dictionary of probs
doctopic = {}
count = 0
for onebag in bow :
    doctopic[count] = model.get_document_topics(onebag)
    count += 1
print ( len(doctopic))

# get the doc list for this iteration



2673


In [191]:
print (doctopic[0])
    

[(1, 0.21935144), (5, 0.045171376), (6, 0.060288794), (9, 0.13064083), (15, 0.088428594), (16, 0.03837406), (21, 0.322497), (27, 0.06970133)]
